In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Convert data to numpy arrays
x_data = data[[ 'open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D



# Convert data to Tensors
x_feature_tensors = torch.tensor(x_data, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data, dtype=torch.float32)




# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

# Make sure to reshape y_train and y_test if they are 1D arrays
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Fit the scaler to the training labels and transform both training and test labels
y_train_scaled = scaler.fit_transform(y_train)
y_val_scaled = scaler.transform(y_val)
y_test_scaled = scaler.transform(y_test)






In [3]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train_scaled) - window_size
batch_size = 1
input_size = x_train_scaled.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [4]:
ö

NameError: name 'ö' is not defined

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available


# Hyperparameter search space
search_space = {
    'learning_rate': [0.00001, 0.00005, 0.0001],
    'window_size': [3, 5, 6],
    'hidden_dim': [128, 256, 512],
    'n_layers': [10, 11, 12],
    'batch_evaluation_frequency': [2, 3, 4, 5]
}

#Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 256,
#                  'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 6463.20556640625}


# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    # Test set evaluation
    with torch.no_grad():
        x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
        y_test_window = torch.tensor(y_test[:params['window_size']], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)

    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()

print(f"Best Parameters: {best_params}")

C:\Users\pitr7\AppData\Local\Temp\ipykernel_2660\1547132306.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_2660\1547132306.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(in

Iteration 1000, Train Loss: 32128.59375, Validation Loss: 45835.42578125
Iteration 2000, Train Loss: 49744.14453125, Validation Loss: 44519.609375
Iteration 3000, Train Loss: 4220.86767578125, Validation Loss: 43346.54296875
Iteration 4000, Train Loss: 5180.62353515625, Validation Loss: 42222.5390625


In [ ]:
# ...

# Test set evaluation
with torch.no_grad():
    x_test_window = torch.tensor(x_test[-params['window_size']:], dtype=torch.float32)
    y_test_window = torch.tensor(y_test[-params['window_size']:], dtype=torch.float32)
    x_test_window = x_test_window.view(1, params['window_size'], input_size)

    hidden = model.init_hidden(1)
    test_outputs, _ = model(x_test_window, hidden)
    test_loss = criterion(test_outputs, y_test_window)

# Print the last entries of the last window
print(f"Last entries of the last window in the test set:")
print("Input values:")
print(x_test_window.squeeze().numpy())
print("Actual output values:")
print(y_test_window.squeeze().numpy())
print("Predicted output values:")
print(test_outputs.squeeze().numpy())

# ...


Last entries of the last window in the test set:
Input values:
[[ 6.0618042e+02  6.1628003e+02  6.0606000e+02  2.1683200e+07
   1.8627701e+01  1.2400000e+00  7.2708064e+08]
 [ 3.1875200e+01  3.1875200e+01  3.1628799e+01  1.0580000e+05
   1.0940000e-01 -8.1999999e-01  7.9405760e+07]
 [ 3.9244801e+01  3.9995201e+01  3.8998402e+01  7.4960000e+05
   2.7750000e-01  9.4999999e-01  1.2576536e+08]
 [ 3.4742401e+01  3.5369598e+01  3.4630402e+01  9.3110000e+05
   1.2240000e-01  2.1700001e+00  1.8372256e+08]
 [ 5.6810040e+02  5.6950043e+02  5.6337958e+02  7.2860000e+06
   1.7679199e+01 -6.6000003e-01  3.5181875e+08]]
Actual output values:
[614.4796  31.6288  39.7488  35.3696 563.9004]
Predicted output values:
83.52607


C:\Users\pitr7\AppData\Local\Temp\ipykernel_1384\2117283741.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_window = torch.tensor(x_test[-params['window_size']:], dtype=torch.float32)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_1384\2117283741.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_window = torch.tensor(y_test[-params['window_size']:], dtype=torch.float32)
